In [ ]:
%load_ext autoreload
%autoreload 2
import pickle
import IPython.display as ipd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from utils.data import *

In [ ]:
folder = 'theorytab'

In [ ]:
path = f'./data/{folder}/'
songs = get_dfs_from_midi(path, min_notes=30, min_gap=0.)
dfs = [item[0] for item in songs]
len(dfs), len(songs)

In [ ]:
# with open(f'./data/dataframes/{folder}/all_songs.pkl', 'wb') as f:
#     pickle.dump(songs, f)
# with open(f'./data/dataframes/{folder}/all_songs.pkl', 'rb') as f:
#     songs = pickle.load(f)

In [ ]:
sample_idx = np.random.randint(len(dfs))
print(f'Artist, title, segment of song: {songs[sample_idx][1:4]}')
print(f'Key of song: {songs[sample_idx][4]}')
print(f'Time signature: {songs[sample_idx][5]}')
print(f'Beats in song: \n {songs[sample_idx][6]}')
midi_data = df_to_midi(songs[sample_idx][0])
fs = 44100
audio_data = midi_data.fluidsynth(fs=fs)
ipd.Audio(audio_data, rate=fs)

# Keys

In [ ]:
n_keys_per_song = [len(item[4]) for item in songs]
plt.hist(n_keys_per_song)

# Percentage of notes within Major scale

In [ ]:
key_to_note = {0: 'C', 1: 'C#', 2: 'D', 3: 'D#', 4: 'E', 5: 'F', 6: 'F#', 7: 'G', 8: 'G#', 9: 'A', 10: 'A#', 11: 'B'}
key_notes = []
for i in range(12):
    key_notes.append(get_notes_from_major_scale(i))

In [ ]:
key = 5
dfs_key = [item[0] for item in songs if item[4][0][0] == key]

In [ ]:
# check percentage of notes in C major scale
key_percentages = {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: [], 10: [], 11: []}
for df in dfs_key:
    for i in range(len(key_notes)):
        percentage = len([note for note in df['Pitch'].values if note in key_notes[i]]) / len(df['Pitch'].values)
        key_percentages[i].append(percentage)

In [ ]:
fig, ax = plt.subplots(4, 3, figsize=(15, 5))
print(f'Key: {key_to_note[key]}')
for i in range(len(key_notes)):
    print(f'Mean percentage of notes in key {key_to_note[i]}: {np.mean(key_percentages[i])}')
    ax[i//3, i%3].hist(key_percentages[i], bins=100)
    ax[i//3, i%3].set_title(f'{key_to_note[i]} major scale')
plt.tight_layout()

# Time Signatures

In [ ]:
n_signatures_per_song = [len(item[5]) for item in songs]
plt.hist(n_signatures_per_song, bins=20)

In [ ]:
beats_per_bar = [item[5][0][0] for item in songs]
plt.hist(beats_per_bar, bins=20)

In [ ]:
note_value = [item[5][0][1] for item in songs]
plt.hist(note_value, bins=20)

In [ ]:
# check that all beats are multiples of the first beat
beats = [item[6] for item in songs]
beats = [item / item[1] for item in beats]
beats = np.concatenate(beats)
beats += 1
beats = beats % beats == 0
beats.all()

In [ ]:
dfs

# Number of notes

In [ ]:
min_length = 30
min_unique_notes = 5
songs = [item for item in songs if len(item[0]) > min_length]
songs = [item for item in songs if item[0]['Pitch'].nunique() > min_unique_notes]
lens = [len(item[0]) for item in songs]
print('Number of songs:', len(songs))
print('Max length:', max(lens))
print('Min length:', min(lens))
print('Mean length:', np.mean(lens))
sorted_lens = sorted(lens)
plt.bar(range(len(sorted_lens)), sorted_lens, width=1.0);

In [ ]:
largest_idx = np.argpartition(lens, -4)[-4:]
print(largest_idx)

In [ ]:
sample_idx = 8677
print(songs[sample_idx][1:4])
midi_data = df_to_midi(songs[sample_idx][0])
fs = 44100
audio_data = midi_data.fluidsynth(fs=fs)
ipd.Audio(audio_data, rate=fs)

In [ ]:
# with open(f'./data/dataframes/{key}/dfs.pkl', 'wb') as f:
with open(f'./data/dataframes/single_key/songs_min_5_unique_notes.pkl', 'wb') as f:
    pickle.dump(songs, f)

In [ ]:
with open(f'./data/dataframes/single_key/songs_min_5_unique_notes.pkl', 'rb') as f:
    songs = pickle.load(f)

# Duration

In [ ]:
# Duration of melodies
durations = []
for item in songs:
    durations.append(item[0]['End'].max())
plt.hist(durations, bins=100);
min(durations), max(durations)

# Pitch range

In [ ]:
# midi_note_df = pd.read_csv('./data/midi_to_notes.csv', index_col=0)
# dfs = [add_octave_and_note(df, midi_note_df) for df in dfs]

In [ ]:
pitch_range([item[0] for item in songs])

In [ ]:
songs2 = trim_by_range(songs, min_range=5, max_range=24)

In [ ]:
pitch_range([item[0] for item in songs2])

In [ ]:
with open(f'./data/dataframes/single_key/songs_30_min_notes_pitch_range_5_24.pkl', 'wb') as f:
    pickle.dump(songs2, f)

In [ ]:
# for df in dfs2:
#     center = (df['Pitch'].max() + df['Pitch'].min()) / 2
#     if center < 45:
#         display(df)
#         temp_df = df.copy()

In [ ]:
# fig, ax = plt.subplots(figsize=(15, 5))
# plt.plot([0], [])

In [ ]:
# dfs3 = move_octaves(dfs2, min_pitch=48, max_pitch=100)
# dfs3 = move_octaves(dfs3, min_pitch=48, max_pitch=80)
dfs3 = move_octaves(dfs2, center_range=[60, 72])

In [ ]:
pitch_range(dfs3)

In [ ]:
with open(f'./data/dataframes/{key}/songs_pitch_range_min_pitch_48_max_pitch_84.pkl', 'wb') as f:
    pickle.dump(dfs3, f)

# Octave translations

In [ ]:
sample_idx = 400
dfs3[sample_idx]

In [ ]:
Fs = 44100
midi_data = df_to_midi(dfs[sample_idx])
audio_data = midi_data.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

In [ ]:
df_octave_down = dfs[sample_idx].copy()
df_octave_down['Pitch'] = df_octave_down['Pitch'] - 12
midi_data = df_to_midi(df_octave_down)
audio_data = midi_data.synthesize(fs=Fs)
ipd.Audio(audio_data, rate=Fs)

# Gaps

In [ ]:
# measure gap between start of next note and end of previous note
neg_gaps = []
pos_gaps = []
for df in dfs:
    gap = df['Start'].iloc[1:].values - df['End'].iloc[:-1].values
    if gap.min() < 0:
        neg_gaps.append(df)
    if gap.max() > 0:
        pos_gaps.append(df)
print(len(neg_gaps), len(pos_gaps))

# Duration + Pitch format

In [ ]:
dfs4 = note_duration_transform(dfs3)

In [ ]:
dfs5, max_pitch = pitch_translation(dfs4)
max_pitch

In [ ]:
pitch_range(dfs5)

In [ ]:
with open(f'./data/dataframes/{key}/dfs_note_dur_offset_{int(max_pitch)}.pkl', 'wb') as f:
    pickle.dump(dfs5, f)

In [ ]:
with open(f'./data/dataframes/{key}/dfs_note_dur_offset_{int(max_pitch)}.pkl', 'rb') as f:
    dfs5 = pickle.load(f)

In [ ]:
dfs5[0]

In [ ]:
dataset = NoteDurationDataset(dfs5, sample_len=20, scale=1., stride=10)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True, num_workers=0, drop_last=True)

In [ ]:
len(dataloader)